In [2]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
google_api_key=os.getenv('GOOGLE_API_KEY') 


In [4]:
from llama_index.core import Settings,StorageContext,load_index_from_storage,VectorStoreIndex,SimpleDirectoryReader


In [5]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
import google.generativeai as genai


c:\Users\Shreyas\Desktop\SmartHire\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
genai.configure(api_key=google_api_key) 


In [8]:
doc=SimpleDirectoryReader(r"C:\Users\Shreyas\Desktop\SmartHire\experiments\data").load_data()


In [9]:
model=Gemini(models='gemini-1.5-pro',api_key=google_api_key)
gemini_embed_model=GeminiEmbedding(model_name='models/text-embedding-004')


In [10]:
Settings.llm = model 
Settings.embed_model=gemini_embed_model
Settings.chunk_size=800
Settings.chunk_overlap=20



In [11]:
index=VectorStoreIndex.from_documents(doc)
index.storage_context.persist()



In [32]:
retriever = index.as_query_engine()
answer=retriever.query("How many years has been it, since he completed his BS, give me an integer answer only [check the logic before you display your answer]")
print(answer.response)



4



In [70]:
parameter="has knowledge of ML?" 


In [71]:
prompt = f"""
    You are an AI system designed to assist in resume screening. 
    Your task is to classify the given parameter into one of three categories:
    
    1. Quantitative: A parameter measured numerically (e.g., years of experience, number of projects, GPA).  
    2. Boolean: A parameter whose answer is a Yes or No ,they usually begin with 'has','knows','is' words.
       (e.g., "Has AWS Certification?", "Knows DevOps?").  
    3. Textual: A parameter requiring knowledge evaluation,skill-assesment or detailed analysis of the entire
       resume, they usually look for 'knowledge','proficiency','relevance' (e.g., "Proficiency in Python","Knowledge in ML).  
    
    Instructions:  
    - Carefully analyze the parameter and determine the correct category.  
    - **Output must be in JSON format**:  
      ```json
      {{ "classification": "Quantitative" }}
      ```
    
    **Input Parameter:** "{parameter}"
    """

In [72]:
response = model.complete(prompt)
print(response)


```json
{ "classification": "Textual" }
```
